<a href="https://colab.research.google.com/github/AYena07/DSL/blob/main/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FIRST function returns set of terminals that can be deducted from non-terminal that we passed to it. Using that fact that we removed left recurtion from our grammar, we can easily implement this function recurcevely, because there is an evidence that we will not stuck in infinite loop. So if we got non-terminal A, we start to iterate throug rules in which left site has only A and if first symbol of rule is terminal h - we simply add it to set. If it is non-terminal B - we get FIRST(B). If it contains '' then we do same operations on next symbol of rule and add FIRST(B) \ {''} to our answer set. If we got to the end of rule and FIRST(last) contains '', then answer will also contain ''.

I will implement function `form_all_first` that takes grammar and returns dict where keys are all non-terminals and values are resutls of applying FIRST to them

In [26]:
def form_all_firsts(G: dict) -> dict:
  result_dict = {}  # will be filled and returned
  tokens = G['toks']
  variables = G['vars']

  def first(non_terminal: str) -> set:
    definition = variables[non_terminal]
    answer_set = set()  # result set of first function
    has_epsilon = False
    for rule in definition:
      if not len(rule):     # case when non-terminal is expiring
        has_epsilon = True
        continue
      for index, symbol in enumerate(rule):
        if symbol in tokens:  # case when first symbol is terminal
          answer_set.add(symbol)
          break
        else:                 # case when first symbol is no-terminal
          if symbol in result_dict:  # we don't need to calculate FIRST if it is already calculated
            firsts_for_another = result_dict[symbol]
            answer_set = set.union(answer_set, firsts_for_another)
          else: # calculating, caching and adding result of FIRST function
            firsts_for_another = first(symbol)
            result_dict[symbol] = firsts_for_another
            answer_set = set.union(answer_set, firsts_for_another)
          if '' not in firsts_for_another:
            break
          elif index == len(rule) - 1:
            has_epsilon = True
    
    if has_epsilon:
      answer_set.add('')
    elif '' in answer_set:
      answer_set.remove('')
    
    return answer_set

  for non_terminal in variables.keys():
    # calculate only if it is not already calculated inside call of FIRST
    if non_terminal not in result_dict:
      result_dict[non_terminal] = first(non_terminal)

  return result_dict

Next is FOLLOW function. The algorithm calculates all terminal symbols that can appear after non-terminal in sentecial dedduction. It calculates once for all, so our function would take grammar and return dict for every non-terminal. 
Starting, we set `follow_dict[S] = {$}`, where $ is the end of input.
For every rule if A -> aB then put follow(A) in follow(B). if A -> aBb then put first(b) in follow(A), where first(seq) is all terminals that can start a word deducted from seq. We repeat these actions in loop because follow sets could change on each iteration, so we repeat this until after passing all rules dict remained unchanged.

In [19]:
def first_for_sequence(first_dict: dict, seq: str, tokens: set):
  answer = set()  #result set
  for symbol in seq:
    if symbol in tokens:  # no sence to continue
      answer.add(symbol)
      return answer
    else:
      answer = set.union(answer, first_dict[symbol])
      if '' not in first_dict[symbol]:
        return answer
      else:
        answer.remove('') # epsilon is not yet in first
  answer.add('')  # if all symbols ->> epsilon, seq ->> epsilon
  return answer

def form_all_follow(G: dict) -> dict:
  result_dict = {}  # will be filled and returned
  tokens = G['toks']
  variables = G['vars']
  hvar = G['hvar']
  updated = True
  epsilon = ''
  first_dict = form_all_firsts(G)

  for non_terminal in variables.keys():
    result_dict[non_terminal] = set()

  result_dict[hvar] = set()
  result_dict[hvar].add('$')
  while updated:
    updated = False
    for non_terminal, definition in variables.items():
      for rule in definition:
        for index, symbol in enumerate(rule):
          if symbol not in tokens:
            first_set = first_for_sequence(first_dict, rule[index + 1:], tokens)
            prev_size = len(result_dict[symbol])
            if epsilon in first_set:
              first_set.remove(epsilon)
              result_dict[symbol] = set.union(result_dict[symbol], first_set)
              result_dict[symbol] = set.union(result_dict[symbol], result_dict[non_terminal])
            else:
              result_dict[symbol] = set.union(result_dict[symbol], first_set)
            new_size = len(result_dict[symbol])
            if new_size != prev_size:
              updated = True
  
  return result_dict


Some Examples

In [28]:
import json
'''
E  -> TR
R  -> +T R| e
T  -> F Y
Y  -> *F Y | e
F  -> (E) | i


Output :
First(E)= { (, i, }
First(R)= { +, e, }
First(T)= { (, i, }
First(Y)= { *, e, }
First(F)= { (, i, }

-----------------------------------------------

Follow(E) = { $, ),  }
Follow(R) = { $, ),  }
Follow(T) = { +, $, ),  }
Follow(Y) = { +, $, ),  }
Follow(F) = { *, +, $, ),  }
'''

GRAMMAR = {
    'toks': set( [
        ('bracket', '('), 
        ('bracket', ')'), 
        ('plus', '+'), 
        ('multiply', '*'),
        ('id', 'i')
    ] ),
    'vars': {
        'E' : [['T', 'R']],
        'R' : [[('plus', '+'), 'T', 'R'],
               []],
        'T' : [['F', 'Y']],
        'Y' : [[('multiply', '*'), 'F', 'Y'],
               []],
        'F' : [[('bracket', '('), 'E', ('bracket', ')')],
               [('id', 'i')]]
    },
    'hvar': 'E'
}
first_dict = form_all_firsts(GRAMMAR)
for key in first_dict.keys():
  first_dict[key] = list(first_dict[key])
follow_dict = form_all_follow(GRAMMAR)
for key in follow_dict.keys():
  follow_dict[key] = list(follow_dict[key])
print(json.dumps(follow_dict, sort_keys=True, indent=2))

{
  "E": [
    "$",
    [
      "bracket",
      ")"
    ]
  ],
  "F": [
    [
      "bracket",
      ")"
    ],
    [
      "multiply",
      "*"
    ],
    "$",
    [
      "plus",
      "+"
    ]
  ],
  "R": [
    "$",
    [
      "bracket",
      ")"
    ]
  ],
  "T": [
    [
      "bracket",
      ")"
    ],
    [
      "plus",
      "+"
    ],
    "$"
  ],
  "Y": [
    [
      "plus",
      "+"
    ],
    "$",
    [
      "bracket",
      ")"
    ]
  ]
}
